In [1]:
import os
import warnings

# disable HF thousand warnings
warnings.simplefilter("ignore")
# set os environ variable for multiprocesses
os.environ["PYTHONWARNINGS"] = "ignore"


In [2]:
from datasets import load_dataset, load_metric

timit = load_dataset("timit_asr", data_dir="../data/timit")

Using custom data configuration default-data_dir=..%2Fdata%2Ftimit
Found cached dataset timit_asr (/home/nm_rostislav/.cache/huggingface/datasets/timit_asr/default-data_dir=..%2Fdata%2Ftimit/0.0.0/43f9448dd5db58e95ee48a277f466481b151f112ea53e27f8173784da9254fb2)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
timit

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 4620
    })
    test: Dataset({
        features: ['file', 'audio', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 1680
    })
})

In [4]:
timit = timit.remove_columns(["dialect_region", "sentence_type", "speaker_id", "id", "word_detail", "text"])

In [5]:
timit

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'phonetic_detail'],
        num_rows: 4620
    })
    test: Dataset({
        features: ['file', 'audio', 'phonetic_detail'],
        num_rows: 1680
    })
})

In [6]:
def join_phones(batch):
    phones_list = batch["phonetic_detail"]["utterance"]
    phones_list.pop(0)
    phones_list.pop(-1)
    phones = " ".join(phones_list)

    return {"phones": phones}

timit = timit.map(join_phones, remove_columns="phonetic_detail")

Loading cached processed dataset at /home/nm_rostislav/.cache/huggingface/datasets/timit_asr/default-data_dir=..%2Fdata%2Ftimit/0.0.0/43f9448dd5db58e95ee48a277f466481b151f112ea53e27f8173784da9254fb2/cache-128177345adeda01.arrow
Loading cached processed dataset at /home/nm_rostislav/.cache/huggingface/datasets/timit_asr/default-data_dir=..%2Fdata%2Ftimit/0.0.0/43f9448dd5db58e95ee48a277f466481b151f112ea53e27f8173784da9254fb2/cache-eb6b0351ac398ad8.arrow


In [7]:
timit

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'phones'],
        num_rows: 4620
    })
    test: Dataset({
        features: ['file', 'audio', 'phones'],
        num_rows: 1680
    })
})

In [8]:
timit["train"][0]["phones"]

'sh ix hv eh dcl jh ih dcl d ah kcl k s ux q en gcl g r ix s ix w ao sh epi w ao dx axr ao l y ih axr'

In [9]:
def extract_all_chars(batch):
    all_text = " ".join(batch["phones"])
    vocab = list(set(all_text.split(" ") + ["|"]))
    return {"vocab": [vocab], "all_text": [all_text]}

vocabs = timit.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=timit.column_names["train"])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [10]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'nx': 0,
 'r': 1,
 'dcl': 2,
 'ay': 3,
 'ix': 4,
 'k': 5,
 'ax-h': 6,
 'ao': 7,
 'jh': 8,
 'l': 9,
 's': 10,
 'w': 11,
 'bcl': 12,
 'ch': 13,
 'hv': 14,
 'gcl': 15,
 'ae': 16,
 'v': 17,
 'b': 18,
 'ey': 19,
 'oy': 20,
 'el': 21,
 'eng': 22,
 'aa': 23,
 'ah': 24,
 'q': 25,
 'dh': 26,
 'sh': 27,
 'uh': 28,
 'em': 29,
 'p': 30,
 'aw': 31,
 'tcl': 32,
 'er': 33,
 'axr': 34,
 'epi': 35,
 'hh': 36,
 'n': 37,
 'dx': 38,
 'ih': 39,
 'ow': 40,
 'g': 41,
 'ng': 42,
 'pau': 43,
 'iy': 44,
 'kcl': 45,
 'ax': 46,
 'pcl': 47,
 'eh': 48,
 'en': 49,
 'th': 50,
 'ux': 51,
 'uw': 52,
 'z': 53,
 'y': 54,
 'zh': 55,
 '|': 56,
 'm': 57,
 't': 58,
 'd': 59,
 'f': 60}

In [11]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))

63


In [12]:
vocab_dict.keys()

dict_keys(['nx', 'r', 'dcl', 'ay', 'ix', 'k', 'ax-h', 'ao', 'jh', 'l', 's', 'w', 'bcl', 'ch', 'hv', 'gcl', 'ae', 'v', 'b', 'ey', 'oy', 'el', 'eng', 'aa', 'ah', 'q', 'dh', 'sh', 'uh', 'em', 'p', 'aw', 'tcl', 'er', 'axr', 'epi', 'hh', 'n', 'dx', 'ih', 'ow', 'g', 'ng', 'pau', 'iy', 'kcl', 'ax', 'pcl', 'eh', 'en', 'th', 'ux', 'uw', 'z', 'y', 'zh', '|', 'm', 't', 'd', 'f', '[UNK]', '[PAD]'])

In [13]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [14]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [15]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1, sampling_rate=16000, 
    padding_value=0.0, do_normalize=True, 
    return_attention_mask=False
)

In [16]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [17]:
import random
import numpy as np

rand_int = random.randint(0, len(timit["train"]))

print("Target text:", timit["train"][rand_int]["phones"])
print("Input array shape:", np.asarray(timit["train"][rand_int]["audio"]["array"]).shape)
print("Sampling rate:", timit["train"][rand_int]["audio"]["sampling_rate"])

Target text: d ih dcl jh ux q iy q y eh tcl
Input array shape: (16794,)
Sampling rate: 16000


In [18]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["phones"]).input_ids
    return batch

In [19]:
timit_prepared = timit.map(prepare_dataset, remove_columns=timit.column_names["train"], num_proc=4)

#0:   0%|          | 0/1155 [00:00<?, ?ex/s]

#1:   0%|          | 0/1155 [00:00<?, ?ex/s]

#2:   0%|          | 0/1155 [00:00<?, ?ex/s]

#3:   0%|          | 0/1155 [00:00<?, ?ex/s]

#0:   0%|          | 0/420 [00:00<?, ?ex/s]

#1:   0%|          | 0/420 [00:00<?, ?ex/s]

#2:   0%|          | 0/420 [00:00<?, ?ex/s]

#3:   0%|          | 0/420 [00:00<?, ?ex/s]

In [20]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [21]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [22]:
wer_metric = load_metric("wer")

In [23]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [24]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base", 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=processor.tokenizer.vocab_size
)

Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForCTC: ['project_hid.weight', 'project_hid.bias', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias', 'quantizer.codevectors', 'project_q.bias', 'project_q.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [25]:
model.freeze_feature_encoder()

In [32]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="wav2vec2-base-timit-psm",
  group_by_length=True,
  per_device_train_batch_size=32,
  evaluation_strategy="steps",
  num_train_epochs=110,
  bf16=True,
  gradient_checkpointing=True, 
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
  dataloader_num_workers=4,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [33]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=timit_prepared["train"],
    eval_dataset=timit_prepared["test"],
    tokenizer=processor.feature_extractor,
)

Using cuda_amp half precision backend


In [34]:
trainer.train()

***** Running training *****
  Num examples = 4620
  Num Epochs = 110
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 96
  Gradient Accumulation steps = 1
  Total optimization steps = 5390


Step,Training Loss,Validation Loss,Wer
500,4.152600,3.570698,1.000000
1000,1.029800,0.524905,0.885904


***** Running Evaluation *****
  Num examples = 1680
  Batch size = 24
Saving model checkpoint to wav2vec2-base-timit-psm/checkpoint-500
Configuration saved in wav2vec2-base-timit-psm/checkpoint-500/config.json
Model weights saved in wav2vec2-base-timit-psm/checkpoint-500/pytorch_model.bin
Feature extractor saved in wav2vec2-base-timit-psm/checkpoint-500/preprocessor_config.json
Deleting older checkpoint [wav2vec2-base-timit-psm/checkpoint-1000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 1680
  Batch size = 24
Saving model checkpoint to wav2vec2-base-timit-psm/checkpoint-1000
Configuration saved in wav2vec2-base-timit-psm/checkpoint-1000/config.json
Model weights saved in wav2vec2-base-timit-psm/checkpoint-1000/pytorch_model.bin
Feature extractor saved in wav2vec2-base-timit-psm/checkpoint-1000/preprocessor_config.json
Deleting older checkpoint [wav2vec2-base-timit-psm/checkpoint-1500] due to args.save_total_limit


KeyboardInterrupt: 